In [1]:
from datasets import load_dataset

ds = load_dataset("Shengtao/recipe")

print(len(ds['train']))

c:\Users\19701\Desktop\MealMate\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\19701\Desktop\MealMate\env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\19701\.cache\huggingface\hub\datasets--Shengtao--recipe. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develope

32722


In [12]:
import openai, os

# api
openai.api_key = ""

def call_chatgpt_api(user_input, model="gpt-4o-mini", system_message="You are a professional chef"):
    response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_input}
            ]
        )
    return response.choices[0].message.content

Load BERT to calculate similarity score

In [27]:
nltk.download('punkt', download_dir='C:/Users/19701/AppData/Roaming/nltk_data')

[nltk_data] Downloading package punkt to
[nltk_data]     C:/Users/19701/AppData/Roaming/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
from bert_score import score
import torch
from transformers import logging as transformers_logging

# Suppress warnings from transformers
transformers_logging.set_verbosity_error()

# Load BERT model and tokenizer for cosine similarity and BERTScore
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for cosine similarity
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# Cosine similarity score between two pieces of text
def cosine_similarity_score(text1, text2):
    emb1 = get_embedding(text1).numpy().reshape(1, -1)
    emb2 = get_embedding(text2).numpy().reshape(1, -1)
    return cosine_similarity(emb1, emb2)[0][0]

# ROUGE scorer setup
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate ROUGE-L score
def rouge_l_score(expected, generated):
    scores = scorer.score(expected, generated)
    return scores['rougeL'].fmeasure

# Function to calculate BERTScore
def bert_score(expected_answers, generated_answers):
    P, R, F1 = score(generated_answers, expected_answers, lang="en", verbose=False)  # Disable verbose logging
    return F1.mean().item()

# Perform benchmarking
def evaluate_benchmark(expected_answers, generated_answers):

    expected = expected_answers
    generated = generated_answers
    print(f"Standard ingredients: {expected}")
    print(f"Generated ingredients: {generated}")

    
    # Cosine Similarity
    cosine_sim = cosine_similarity_score(expected, generated)
    print(f"Cosine Similarity: {cosine_sim:.4f}")
    
    # ROUGE-L
    rouge_l = rouge_l_score(expected, generated)
    print(f"ROUGE-L: {rouge_l:.4f}")
    
    # BERTScore
    bertscore_f1 = bert_score([expected], [generated])
    print(f"BERTScore F1: {bertscore_f1:.4f}")

    print("\n")


# Run the evaluation


In [34]:
titles = ds['train']['title']
ingredients = ds['train']['ingredients']

for i in range(2):
    title = titles[i]
    query = f"What's the ingredients of {title}? Please list the specific amount of each ingredient only, separate with semicolons."
    print(f"Query: {query}")
    
    ## TODO: link to the chef de cuisine
    response = call_chatgpt_api(query)
    ingredient = ingredients[i]

    
    
    evaluate_benchmark(ingredient, response)

Query: What's the ingredients of Simple Macaroni and Cheese? Please list the specific amount of each ingredient only, separate with semicolons.
Standard ingredients: 1 (8 ounce) box elbow macaroni ; ¼ cup butter ; ¼ cup all-purpose flour ; ½ teaspoon salt ;   ground black pepper to taste ; 2 cups milk ; 2 cups shredded Cheddar cheese
Generated ingredients: 8 ounces elbow macaroni; 2 tablespoons butter; 2 tablespoons all-purpose flour; 2 cups milk; 2 cups shredded sharp cheddar cheese; 1/2 teaspoon salt; 1/4 teaspoon black pepper; 1/4 teaspoon paprika (optional).
Cosine Similarity: 0.9480
ROUGE-L: 0.5161
BERTScore F1: 0.9071


Query: What's the ingredients of Gourmet Mushroom Risotto? Please list the specific amount of each ingredient only, separate with semicolons.
Standard ingredients: 6 cups chicken broth, divided ; 3 tablespoons olive oil, divided ; 1 pound portobello mushrooms, thinly sliced ; 1 pound white mushrooms, thinly sliced ; 2  shallots, diced ; 1 ½ cups Arborio rice ; ½ c

In [24]:
import nltk

print(nltk.data.path)

['C:\\Users\\19701/nltk_data', 'c:\\Users\\19701\\Desktop\\MealMate\\env\\nltk_data', 'c:\\Users\\19701\\Desktop\\MealMate\\env\\share\\nltk_data', 'c:\\Users\\19701\\Desktop\\MealMate\\env\\lib\\nltk_data', 'C:\\Users\\19701\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']
